In [22]:
import os 
from super_gradients.training import Trainer
from super_gradients.training.dataloaders.dataloaders import (
    coco_detection_yolo_format_train, coco_detection_yolo_format_val)
from super_gradients.training import models
from super_gradients.training.losses import PPYoloELoss
from super_gradients.training.metrics import DetectionMetrics_050
from super_gradients.training.models.detection_models.pp_yolo_e import PPYoloEPostPredictionCallback
from dotenv import load_dotenv
import cv2 
import supervision as sv


In [23]:
!nvidia-smi # check GPU availability 


Wed Jun 19 15:06:37 2024       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 510.47.03    Driver Version: 510.47.03    CUDA Version: 11.6     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ...  On   | 00000000:17:00.0  On |                  N/A |
|  0%   40C    P8    21W / 320W |    258MiB / 10240MiB |     17%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [36]:
# Declare paths
HOME = os.getcwd()
LOCATION = f'{HOME}/vehicles'
CHECKPOINT_DIR = f'{HOME}/checkpoints'

print(LOCATION)

/home/lpalombi/Desktop/YOLONAS_Training/vehicles


In [41]:
# Declare constraints 
MODEL_ARCH = os.getenv('MODEL_ARCH') # model architecture load from .env (yolo nas s)
EXPERIMENT_NAME = 'vehicles' # name of experiment (data folder)
CLASSES = ['0'] # Array of class label names
BATCH_SIZE = 8 # # of batches
MAX_EPOCHS = 25 # # of epochs

In [26]:
from super_gradients.training import Trainer

trainer = Trainer(experiment_name=EXPERIMENT_NAME, ckpt_root_dir=CHECKPOINT_DIR)

In [30]:
dataset_params = {
    'data_dir': LOCATION,
    'train_images_dir':'images/train',
    'train_labels_dir':'labels/train',
    'val_images_dir':'images/val',
    'val_labels_dir':'labels/val',
    'test_images_dir':'images/test',
    'test_labels_dir':'labels/test',
    'classes': CLASSES
}

In [45]:
from super_gradients.training.dataloaders.dataloaders import (
    coco_detection_yolo_format_train, coco_detection_yolo_format_val)

train_data = coco_detection_yolo_format_train(
    dataset_params={
        'show_all_warnings': True,
        'data_dir': dataset_params['data_dir'],
        'images_dir': dataset_params['train_images_dir'],
        'labels_dir': dataset_params['train_labels_dir'],
        'classes': dataset_params['classes']
    },
    dataloader_params={
        'batch_size': BATCH_SIZE,
        'num_workers': 2
    }
)

# val_data = coco_detection_yolo_format_val(
#     dataset_params={
#         'data_dir': dataset_params['data_dir'],
#         'images_dir': dataset_params['val_images_dir'],
#         'labels_dir': dataset_params['val_labels_dir'],
#         'classes': dataset_params['classes']
#     },
#     dataloader_params={
#         'batch_size': BATCH_SIZE,
#         'num_workers': 2
#     }
# )

# test_data = coco_detection_yolo_format_val(
#     dataset_params={
#         'data_dir': dataset_params['data_dir'],
#         'images_dir': dataset_params['test_images_dir'],
#         'labels_dir': dataset_params['test_labels_dir'],
#         'classes': dataset_params['classes']
#     },
#     dataloader_params={
#         'batch_size': BATCH_SIZE,
#         'num_workers': 2
#     }
# )

[2024-06-19 15:26:29] WARNING - yolo_format_detection.py - 10 images are note associated to any label file
[2024-06-19 15:26:29] WARNING - yolo_format_detection.py - As a consequence, 432/442 images and 432/432 label files will be used.
[2024-06-19 15:26:29] INFO - detection_dataset.py - Dataset Initialization in progress. `cache_annotations=True` causes the process to take longer due to full dataset indexing.
Indexing dataset annotations:   0%|          | 0/432 [00:00<?, ?it/s][2024-06-19 15:26:29] WARNING - yolo_format_detection.py - Line `0 0.5031122259383128 0.0 0.5536858974358975 0.0 0.5536858974358975 0.09544159544159543 0.5031122259383128 0.09544159544159543
` of file /home/lpalombi/Desktop/YOLONAS_Training/vehicles/labels/train/ef0958863019b1703dfcd83208ef7eed.txt will be ignored because not cannot be parsed to (label, cx, cy, w, h) format, with Exception:
too many values to unpack (expected 5)
[2024-06-19 15:26:29] WARNING - yolo_format_detection.py - Line `0 0.618589743589743

EmptyDatasetException: Empty Dataset: Out of 432 images, not a single one was found with any of these classes: None